In [1]:
using Revise 

In [2]:
using LazySets
using ModelVerification

In [3]:
using PyCall
using CSV
using ONNX
using Flux
include("/home/verification/ModelVerification.jl/vnnlib_parser.jl")

read_vnnlib_simple (generic function with 1 method)

In [4]:
function onnx_to_nnet(onnx_file)
    pushfirst!(PyVector(pyimport("sys")."path"), @__DIR__)
    nnet = pyimport("NNet")
    use_gz = split(onnx_file, ".")[end] == "gz"
    if use_gz
        onnx_file = onnx_file[1:end-3]
    end
    nnet_file = onnx_file[1:end-4] * "nnet"
    isfile(nnet_file) && return
    nnet.onnx2nnet(onnx_file, nnetFile=nnet_file)
end

function verify_an_instance(onnx_file, spec_file, timeout)
    use_gz = split(onnx_file, ".")[end] == "gz"
    nnet_file = use_gz ? onnx_file[1:end-7] * "nnet" : onnx_file[1:end-4] * "nnet"
    net = ModelVerification.read_nnet(nnet_file)
    n_in = size(net.layers[1].weights)[2]
    n_out = length(net.layers[end].bias)
    flux_model = Flux.Chain(net)
    specs = read_vnnlib_simple(spec_file, n_in, n_out)
    current_time = 0
    for spec in specs
        X_range, Y_cons = spec
        lb = [bd[1] for bd in X_range]
        ub = [bd[2] for bd in X_range]
        X = Hyperrectangle(low = lb, high = ub)
        res = nothing
        A = []
        b = []
        for Y_con in Y_cons
            A = hcat(Y_con[1]...)'
            b = Y_con[2]
            Y = HPolytope(-A, -b)
            search_method = BFS(max_iter=1e5, batch_size=4)
            split_method = Bisect(1)
            prop_method = Ai2z()
            problem = Problem(flux_model, X, Y)
            res = @timed verify(search_method, split_method, prop_method, problem)
            current_time += res.time 
            if(current_time >= timeout)
                print(111111111111)
                print("\n")
                print(current_time)
                print("\n")
                print(111111111)
                print("\n")
                return "unknown"
            end

            res.value.status == :violated && (return "violated")
            res.value.status == :unknown && (return "unknown")
        end
    end
    return "holds"
end

function main(args)
    file = CSV.File(args, header=false)
    dirpath = args[1:end-20]      
    outpath = dirpath * "out_MV.txt"
    result_file = open("/home/verification/ModelVerification.jl/output_MV.txt", "w")
    all_time = 0
    ave_time = 0
    instance_num = -1
    max_time = 0
    min_time = Inf
    hold_number = 0
    max_memory = 0
    all_memory = 0
    ave_memory = 0
    for row in file
        instance_num += 1
        onnx_file = dirpath * row[1] 
        vnnlib_file = dirpath * row[2]
        timeout = row[3]
        onnx_to_nnet(onnx_file)
        result = @timed verify_an_instance(onnx_file, vnnlib_file, timeout)
        print(instance_num)
        print("\n")
        print(result)
        print("\n") 
        all_time += result.time
        all_memory += result.bytes 
        if result.time > max_time
            max_time = result.time
        elseif result.time < min_time
            min_time = result.time
        end

        if result.bytes > max_memory
            max_memory = result.bytes
        end

        if(result.value === "holds")
            hold_number += 1
        end

        text = "This is instance $instance_num.\n"
        write(result_file, text)
        write(result_file, result.value)
        write(result_file, "\n")
    end
    ave_time = all_time / (instance_num + 1)
    ave_memory = all_memory / (instance_num + 1)
    print(ave_time)
    print("\n")
    print(max_time)
    print("\n")
    print(min_time)
    print("\n")
    print(hold_number)
    print("\n")
    text = "average time is $ave_time.\n"
    write(result_file, text)
    write(result_file, "\n")
    text = "max time is $max_time.\n"
    write(result_file, text)
    write(result_file, "\n")
    text = "min time is $min_time.\n"
    write(result_file, text)
    write(result_file, "\n")
    text = "holds number is $hold_number.\n"
    write(result_file, text)
    write(result_file, "\n")
    text = "average bytes is $ave_memory.\n"
    write(result_file, text)
    write(result_file, "\n")
    text = "max bytes is $max_memory.\n"
    write(result_file, text)
    write(result_file, "\n")
    #close(file)
    close(result_file)
end

main("/home/verification/vnncomp2021/benchmarks/acasxu/acasxu_instances.csv")

0


(value

 = "

ho

lds", time = 71.758668519

...


(value = "holds", time = 1.702393597, bytes = 331592597, gctime = 0.084376559, gcstats = Base.GC_Diff(331592597, 273, 0, 458285, 53140, 404, 84376559, 1, 0))
126
(value = "violated", time = 0.008442457, bytes = 3121607, gctime = 0.0, gcstats = Base.GC_Diff(3121607, 16, 0, 6960, 277, 0, 0, 0, 0))
127
(value = "violated", time = 0.009316393, bytes = 3004356, gctime = 0.0, gcstats = Base.GC_Diff(3004356, 11, 0, 6951, 282, 0, 0, 0, 0))
128
(value = "violated", time = 0.02476924, bytes = 3101459, gctime = 0.0, gcstats = Base.GC_Diff(3101459, 16, 0, 6960, 277, 0, 0, 0, 0))


129
(value = "violated", time = 0.13166003, bytes = 3018354, gctime = 0.09685123, gcstats = Base.GC_Diff(3018354, 11, 0, 6951, 282, 167, 96851230, 1, 0))
130
(value = "violated", time = 0.044844232, bytes = 3303083, gctime = 0.0, gcstats = Base.GC_Diff(3303083, 21, 0, 6969, 272, 0, 0, 0, 0))
131
(value = "violated", time = 0.027525407, bytes = 3332264, gctime = 0.0, gcstats = Base.GC_Diff(3332264, 21, 0, 6968, 273, 0, 0, 0, 0))


132
(value = "holds", time = 0.075298361, bytes = 15034453, gctime = 0.0, gcstats = Base.GC_Diff(15034453, 12, 0, 20909, 2303, 0, 0, 0, 0))
111111111111


527.953886243

InterruptException: Error trying to display an error.